In [1]:
import os
import re

In [2]:
# Directory to search
#directory_to_search = "/home/elias/IdeaProjects/systemds/scripts/builtin"
directory_to_search = "/Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin"

matches = []

# Regex pattern for matching function and return statements
pattern = re.compile(r"(?P<func>\b\w+\b\s*=\s*function\s*\([^)]*\)\s*)"  # Match 'fname = function (...)'
                     r"(?:(?P<return>return\s*\([^)]*\)))?",             # Match 'return (...)', optional
                     re.DOTALL)                                          # Enable dotall to match across lines

# Walk through the directory
for root, _, files in os.walk(directory_to_search):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

                # Find all matches in the file
                no_match = True
                for match in pattern.finditer(content):
                    function_header = match.group()
                    matches.append({
                        "file_name": file_path,
                        "function_header": function_header
                    })
                    no_match = False
                if no_match:
                    print(file_path)
        except (UnicodeDecodeError, IOError):
            print(f"Could not read file: {file_path}")
            
results = matches

In [3]:
len([1 for _ in files for _, _, files in os.walk(directory_to_search)])

190

In [4]:
len(results)

432

In [5]:
results_inner = [r for r in results if r["function_header"][1] != "_"]
results  = [r for r in results if r["function_header"][1] == "_"]
len(results)

201

In [6]:
for s in [r["function_header"][:15] for r in results_inner]:
    print(s)

matrixToList = 
applyDataFlag =
confirmData = f
dummycoding = f
dummycodingAppl
imputeByFd = fu
imputeByFdApply
forward_fill = 
SMOTE  = functi
fillDefault = f
fillDefaultAppl
storeDataForPru
flipLabels = fu
colMinMax = fun
isFD = function
colDistinct = f
parallelBinaryS
predict_TT = fu
predict_GEMM = 
createTTNodeTen
createGEMMNodeT
calculateDistan
sortAndGetK = f
getInvalidsMask
fStat_tailprob 
sqrt_failsafe =
linear_regressi
reorder_matrix 
checkAIC = func
calcKnotsDerivK
interpSplineCG 
CGSolver = func
truncCG = funct
resizeCG = func
fix_outliers_sd
getColMean_Sd =
getOutputValueF
calculateTreesO
calculateNewPre
getOutputValueF
buildOneTreeReg
buildOneTreeCla
calculateChildN
addOutputRow = 
getTypeOfFeatur
updateMatrices 
calcAvailableRo
dataQueuePop = 
dataQueuePush =
findBestFeature
findBestSplit =
splitMatrixByVa
splitMatrixByVa
splitMatrixByCa
calculateSimila
calculateSimila
calculateOutput
calculateOutput
calculateResidu
orderFeature = 
average = funct
filterOutliers 
sqrtDiag

In [7]:
# Print results
for result in results:
    print(f"File: {result['file_name']}")
    print(f"Function Header: {result['function_header']}")
    print("-" * 40)

File: /Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin/mse.dml
Function Header: m_mse = function(Matrix[Double] X, Matrix[Double] Y, Matrix[Double] P = matrix(0, rows=0, cols=0)
----------------------------------------
File: /Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin/executePipeline.dml
Function Header: f_executePipeline = function(Frame[String] pipeline, Matrix[Double] Xtrain,  Matrix[Double] Ytrain, 
  Matrix[Double] Xtest,  Matrix[Double] Ytest, List[Unknown] metaList, Matrix[Double] hyperParameters, Matrix[Double] hpForPruning = as.matrix(0)
----------------------------------------
File: /Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin/executePipeline.dml
Function Header: m_normalize = function(Matrix[Double] X)
  return (Matrix[Double] Y, Matrix[Double] cmin, Matrix[Double] cmax)
----------------------------------------
File: /Users/eliasstrauss/Desktop/TU/systemds-fork/scripts/builtin/discoverFD.dml
Function Header: m_discoverFD = functio

In [8]:
unique_parameters = dict()
intput_parameter_filter = re.compile(r"function\s*\(([^)]*)\)")
param_with_default_pattern = re.compile(r"(\w+)\s+(\w+)\s*=\s*(.+)")
for result in results:
    function_header = result["function_header"]

    # Search for the parameters inside "function(...)"
    param_match = intput_parameter_filter.search(function_header)
    if param_match:
        param_str = param_match.group(1)  # The content inside the parentheses

        # Split the parameters by commas and process each one
        params = [p.strip() for p in param_str.split(",") if p.strip()]
        for param in params:
            if not '=' in param:
                parts = param.split()
                if not parts[-1] in unique_parameters:
                    unique_parameters[parts[-1]] = 1
                else:
                    unique_parameters[parts[-1]] += 1
            else:
               # Extract the parameter name from possible formats:
                # NAME, DATATYPE NAME, DATATYPE NAME=DEFAULT, DATATYPE NAME = DEFAULT
                match = param_with_default_pattern.match(param)
                if match:
                    param_name = match.group(2)
                    if not param_name in unique_parameters:
                        unique_parameters[param_name] = 1
                    else:
                        unique_parameters[param_name] += 1
                else:
                    print("Warning no match for: " + param)

Warning no match for: Matrix[Double] P = matrix(0
Warning no match for: rows=0
Warning no match for: cols=0
Warning no match for: Matrix[Double] hpForPruning = as.matrix(0
Warning no match for: Matrix[Double] P = matrix(0
Warning no match for: rows=0
Warning no match for: cols=0
Warning no match for: Matrix[Double] y = matrix(0
Warning no match for: Matrix[Double] Y = matrix(0.0
Warning no match for: rows=1
Warning no match for: cols=1
Warning no match for: Matrix[Double] ytest = matrix(0
Warning no match for: Matrix[Double] R = matrix(1
Warning no match for: rows=1
Warning no match for: cols=nrow(X
Warning no match for: Matrix[Double] P = matrix(0
Warning no match for: rows=0
Warning no match for: cols=0
Warning no match for: Matrix[Double] P = matrix(0
Warning no match for: rows=0
Warning no match for: cols=0
Warning no match for: Frame[Unknown] dataTest = as.frame("NULL"
Warning no match for: Matrix[Double] P = matrix(0
Warning no match for: rows=0
Warning no match for: cols=0
Warni

In [9]:
# manually handle exceptions
# unique_parameters.add("numConfigs")
# unique_parameters.add("eta")
# unique_parameters.add("k_min")

In [10]:
len(unique_parameters)

342

In [11]:
names = list(unique_parameters.keys())
names.sort()

In [12]:
ml = max([len(name) for name in names])

In [13]:
ml

24

In [14]:
for name in names:
    print("{name: <28} : {c}".format(name=name, c=unique_parameters[name]))

0                            : 10
1                            : 4
A                            : 3
B                            : 4
C                            : 6
CL                           : 1
CL_T                         : 1
Centering                    : 1
Clusters                     : 2
D                            : 4
F                            : 2
F1                           : 1
G                            : 3
GI                           : 1
Graph                        : 1
H                            : 2
Hin                          : 2
I                            : 2
IQR                          : 1
K                            : 3
L                            : 2
LHSfeatures                  : 1
LHSthreshold                 : 1
M                            : 8
Mask                         : 1
P                            : 9
Q                            : 2
Q1                           : 1
Q3                           : 1
R                            : 7
RHSfeatur

In [15]:
para_by_count = sorted([pair for pair in unique_parameters.items()], key=lambda pair: pair[1], reverse=True)

In [16]:
for n, c in para_by_count:
    print("{name: <28} : {c}".format(name=n, c=c))

X                            : 131
verbose                      : 45
Y                            : 36
img_in                       : 19
tol                          : 17
seed                         : 16
reg                          : 15
maxIter                      : 15
y                            : 14
k                            : 14
threshold                    : 10
0                            : 10
fill_value                   : 10
mask                         : 9
P                            : 9
intercept                    : 8
M                            : 8
maxi                         : 7
R                            : 7
C                            : 6
model                        : 6
rank                         : 5
repairMethod                 : 5
d                            : 5
s_cols                       : 5
s_rows                       : 5
alpha                        : 5
w                            : 4
B                            : 4
method                       